# Lab E.2 Solutions: GCN from Scratch

Complete solutions to all exercises in Lab E.2.

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

## Exercise 1 Solution: Different Hidden Dimensions

In [ ]:
class GCN(nn.Module):
    def __init__(self, num_features, num_classes, hidden_dim=64):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

def train_and_eval(hidden_dim, epochs=200):
    model = GCN(dataset.num_features, dataset.num_classes, hidden_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    start = time.time()
    best_acc = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            pred = model(data.x, data.edge_index).argmax(dim=1)
            acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()
            if acc > best_acc:
                best_acc = acc
    
    train_time = time.time() - start
    n_params = sum(p.numel() for p in model.parameters())
    
    return best_acc, n_params, train_time

# Compare hidden dimensions
hidden_dims = [16, 32, 64, 128, 256]
results = []

print("Comparing Hidden Dimensions")
print("=" * 60)
print(f"{'Hidden Dim':<12} {'Test Acc':<12} {'Params':<15} {'Time (s)':<10}")
print("-" * 60)

for hidden_dim in hidden_dims:
    acc, params, t = train_and_eval(hidden_dim)
    results.append((hidden_dim, acc, params, t))
    print(f"{hidden_dim:<12} {acc:<12.4f} {params:<15,} {t:<10.2f}")

# Find best
best = max(results, key=lambda x: x[1])
print("\n" + "=" * 60)
print(f"🏆 Best: hidden_dim={best[0]} with {best[1]:.4f} accuracy")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

dims = [r[0] for r in results]
accs = [r[1] for r in results]
params = [r[2] for r in results]
times = [r[3] for r in results]

axes[0].bar(range(len(dims)), accs, color='steelblue')
axes[0].set_xticks(range(len(dims)))
axes[0].set_xticklabels(dims)
axes[0].set_xlabel('Hidden Dimension')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Accuracy vs Hidden Dim')

axes[1].bar(range(len(dims)), [p/1000 for p in params], color='coral')
axes[1].set_xticks(range(len(dims)))
axes[1].set_xticklabels(dims)
axes[1].set_xlabel('Hidden Dimension')
axes[1].set_ylabel('Parameters (thousands)')
axes[1].set_title('Parameters vs Hidden Dim')

axes[2].bar(range(len(dims)), times, color='seagreen')
axes[2].set_xticks(range(len(dims)))
axes[2].set_xticklabels(dims)
axes[2].set_xlabel('Hidden Dimension')
axes[2].set_ylabel('Training Time (s)')
axes[2].set_title('Time vs Hidden Dim')

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("   - Accuracy plateaus around hidden_dim=64")
print("   - Parameters grow quadratically with hidden_dim")
print("   - Larger isn't always better - diminishing returns!")

## Exercise 2 Solution: 3-Layer GCN

In [ ]:
class GCN3Layer(nn.Module):
    """Three-layer GCN - watch out for over-smoothing!"""
    
    def __init__(self, num_features, num_classes, hidden_dim=64):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, num_classes)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        return x
    
    def get_embeddings(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x

# Train 2-layer and 3-layer for comparison
def train_model(Model, epochs=200):
    model = Model(dataset.num_features, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            pred = model(data.x, data.edge_index).argmax(dim=1)
            acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()
            if acc > best_acc:
                best_acc = acc
    
    return model, best_acc

print("Comparing 2-Layer vs 3-Layer GCN")
print("=" * 50)

_, acc_2layer = train_model(GCN)
model_3layer, acc_3layer = train_model(GCN3Layer)

print(f"2-Layer GCN: {acc_2layer:.4f}")
print(f"3-Layer GCN: {acc_3layer:.4f}")

if acc_3layer < acc_2layer:
    print("\n⚠️ 3-layer is WORSE! This could be over-smoothing.")
else:
    print("\n✅ 3-layer is better or equal.")

## Challenge Solution: ResGCN

In [ ]:
class ResGCN(nn.Module):
    """GCN with residual connections to fight over-smoothing."""
    
    def __init__(self, num_features, num_classes, hidden_dim=64, num_layers=4):
        super().__init__()
        
        # First layer (dimension change)
        self.conv_first = GCNConv(num_features, hidden_dim)
        
        # Middle layers (residual)
        self.convs = nn.ModuleList([
            GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 2)
        ])
        
        # Last layer (classification)
        self.conv_last = GCNConv(hidden_dim, num_classes)
        
        self.dropout = 0.5
    
    def forward(self, x, edge_index):
        # First layer (no residual - dimension change)
        x = F.relu(self.conv_first(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Middle layers with residual connections
        for conv in self.convs:
            residual = x  # Save for skip connection
            x = conv(x, edge_index)
            x = x + residual  # Residual connection!
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Last layer
        x = self.conv_last(x, edge_index)
        return x

# Compare different depths
print("Comparing GCN Depths with/without Residuals")
print("=" * 60)

for num_layers in [2, 4, 6, 8]:
    # Standard deep GCN
    class DeepGCN(nn.Module):
        def __init__(self):
            super().__init__()
            self.convs = nn.ModuleList()
            self.convs.append(GCNConv(dataset.num_features, 64))
            for _ in range(num_layers - 2):
                self.convs.append(GCNConv(64, 64))
            self.convs.append(GCNConv(64, dataset.num_classes))
        
        def forward(self, x, edge_index):
            for i, conv in enumerate(self.convs[:-1]):
                x = F.relu(conv(x, edge_index))
                x = F.dropout(x, p=0.5, training=self.training)
            return self.convs[-1](x, edge_index)
    
    # Train both
    _, acc_standard = train_model(DeepGCN)
    
    # ResGCN with same depth
    class ResGCNFixed(ResGCN):
        def __init__(self, *args, **kwargs):
            super().__init__(dataset.num_features, dataset.num_classes, 
                           hidden_dim=64, num_layers=num_layers)
    
    _, acc_res = train_model(ResGCNFixed)
    
    print(f"Layers={num_layers}: Standard={acc_standard:.4f}, ResGCN={acc_res:.4f}")

print("\n💡 Residual connections help with deeper networks!")

---

## Key Takeaways

1. **Hidden dimension ~64 is often sufficient** for citation networks
2. **More layers can hurt** due to over-smoothing (nodes become too similar)
3. **Residual connections help** with deeper networks
4. **2-3 layers is the sweet spot** for most GNN tasks